In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --execute-command "pwsh -c `"../../../../scripts/invoke-dib.ps1 test.dib`"" } | Invoke-Block -Retries 5

00:00:00 #1 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #2 [Debug] executeAsync / options: { Command =
   "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:main@464-1020> }
00:00:00 #3 [Verbose] > pwd: /home/runner/work/polyglot/polyglot/apps/spiral/temp/test
00:00:00 #4 [Verbose] > dll_path: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:00 #5 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #6 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:00 #7 [Verbose] testPortOpen / ex: System.AggregateException

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(GetExecutableSuffix) test.dib spi } | Invoke-Block

00:00:00 #1 [Debug] writeDibCode / output: Spi / path: test.dib
00:00:00 #2 [Debug] parseDibCode / output: Spi / file: test.dib


In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --build-file test.spi test.fsx --timeout 10000 } | Invoke-Block

00:00:00 #1 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #2 [Debug] executeAsync / options: { Command =
   "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:main@425-297> }
00:00:00 #3 [Verbose] > pwd: /home/runner/work/polyglot/polyglot/apps/spiral/temp/test
00:00:00 #4 [Verbose] > dll_path: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:00 #5 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #6 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:00 #7 [Verbose] testPortOpen / ex: System.AggregateException:

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.14.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @7sharp9
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
.> dotnet restore test.fable-temp.csproj -p:FABLE_COMPILER=true -p:FABLE_COMPILER_4=true -p:FABLE_COMPILER_RUST=true
  Determining projects to restore...
  Paket version 8.1.0-alpha001+27cb9f111f156e8b68d08dc293857425c0ade23d
  The last full restore is still up to date. Nothing left to do.
  Total time taken: 0 milliseconds
  Paket version 8.1.0-alpha001+27cb9f111f156e8b68d08dc293857425c0ade23d
  Restoring /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.fable-temp.csproj
  Starting restore process.
  Total time taken: 0 milliseconds
  Restored /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.fable-temp.csproj (in 240 ms).
.> dotnet restore /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.fsproj
  Determining projects to restore...
  Restored /home/runner/work/polyglot/polyglot/apps/

In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
| Set-Content test.rs

In [ ]:
cargo fmt --

In [ ]:
cargo build --release

   Compiling cfg-if v1.0.0
   Compiling libc v0.2.153
   Compiling autocfg v1.1.0
   Compiling libm v0.2.8
   Compiling proc-macro2 v1.0.79
   Compiling num-traits v0.2.18
   Compiling unicode-ident v1.0.12
   Compiling quote v1.0.35
   Compiling getrandom v0.2.12
   Compiling once_cell v1.19.0
   Compiling memchr v2.7.1
   Compiling rustix v0.38.31
   Compiling rand_core v0.6.4
   Compiling syn v2.0.53
   Compiling linux-raw-sys v0.4.13
   Compiling bitflags v2.4.2
   Compiling regex-syntax v0.8.2
   Compiling aho-corasick v1.1.2
   Compiling tracing-core v0.1.32
   Compiling num-bigint v0.4.4
   Compiling lazy_static v1.4.0
   Compiling fastrand v2.0.1
   Compiling rust_decimal v1.34.3
   Compiling ppv-lite86 v0.2.17
   Compiling tempfile v3.10.1
   Compiling regex-automata v0.4.6
   Compiling rand_chacha v0.3.1
   Compiling num-integer v0.1.46
   Compiling wait-timeout v0.2.0
   Compiling thiserror v1.0.58
   Compiling log v0.4.21
   Compiling overload v0.1.1
   Compiling arrayvec v

In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling cfg-if v1.0.0
   Compiling libc v0.2.153
   Compiling libm v0.2.8
   Compiling memchr v2.7.1
   Compiling once_cell v1.19.0
   Compiling regex-syntax v0.8.2
   Compiling getrandom v0.2.12
   Compiling num-traits v0.2.18
   Compiling rand_core v0.6.4
   Compiling bitflags v2.4.2
   Compiling linux-raw-sys v0.4.13
   Compiling aho-corasick v1.1.2
   Compiling rustix v0.38.31
   Compiling tracing-core v0.1.32
   Compiling fastrand v2.0.1
   Compiling ppv-lite86 v0.2.17
   Compiling lazy_static v1.4.0
   Compiling rand_chacha v0.3.1
   Compiling tempfile v3.10.1
   Compiling regex-automata v0.4.6
   Compiling num-integer v0.1.46
   Compiling wait-timeout v0.2.0
   Compiling quick-error v1.2.3
   Compiling log v0.4.21
   Compiling iana-time-zone v0.1.60
   Compiling arrayvec v0.7.4
   Compiling overload v0.1.1
   Compiling bit-vec v0.6.3
   Compiling fnv v1.0.7
   Compiling rusty-fork v0.3.0
   Compiling nu-ansi-term v0.46.0
   Compiling bit-set v0.5.3
   Compiling rust_decimal

In [ ]:
{ . $ScriptDir/../../../../target/release/spiral_temp_test$(GetExecutableSuffix) } | Invoke-Block

app=test
